# **Ejercicio 2 - Preprocesado**

Este es el notebook del Ejercicio 2 de la práctica de NLP. He dividido la práctica en los siguientes puntos.

- Carga de librerías
- Funciones utilizadas
- Carga de datasets
- Pre-preprocesado y Stop Words
- Normalización sin lemmatización
- Lemmatización
- Preprocesado de los datasets

## Carga de librerías

Cargamos las librerías que necesitamos para este notebook, que no incluyen las relacionadas con el modelado.

In [1]:
import random
import numpy as np
import pandas as pd

# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.feature_selection import chi2
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

import matplotlib.pyplot as plt

Sí que incluimos las necesarias para el prepocesado, tales como num2words y stop_words.

In [2]:
import re
import string
import unicodedata

In [3]:
from bs4 import BeautifulSoup
import re

In [4]:
!pip install num2words
from num2words import num2words

In [6]:
!pip install stop_words
from stop_words import get_stop_words

Instalamos Spacy. Lo que ocurre en este caso es que nos pide reiniciar el kernel, por lo que tenemos que comenzar desde el principio. A la segunda vez, la instalación se completa sin problemas.

In [5]:
!pip install spacy==3.7.2

Bajamos un modelo simple de SpaCy de la lengua inglesa.

In [7]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_web_sm: filename=en_core_web_sm-3.7.1-py3-none-any.whl size=12803371 sha256=e7ea8ebeae501bdb8ffc06dfe7dc2169339bc6c75b210b12fe24681a03ffdc3c
  Stored in directory: /root/.cache/pip/wheels/f9/64/b3/10bb1b7e2edd8a3e833cf7805de070e7a92e11eb5ee5f89263
Successfully built en_core_web_sm
  Attempting uninstall: en_core_web_sm
    Found existing installation: en_core_web_sm 3.8.0
    Uninstalling en_core_web_sm-3.8.0:
      Successfully uninstalled en_core_web_sm-3.8.0


Importamos el modelo, y comprobamos el pipeline que contiene, que incluye un lemmatizador.

In [8]:
import en_core_web_sm

nlp_en = en_core_web_sm.load()
nlp_en.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x792205b2b050>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x792205b2b470>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x792205b8a570>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x792205835490>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7922057f7c10>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x792205b8a650>)]

## Funciones utilizadas

Soy consciente que el enunciado pide una única función de preprocesado, pero pienso que es mejor el dividir este proceso en dos: una función que excluya la lemmatización, y otra que se centra en lemmatización. Nos extenderemos algo más en ellas en los apartados correspondientes.

Otra razón adicional es que no he conseguido el integrar la lemmatización de Spacy en la función de normalización.

El caso es que la siguiente es la función de normalización, text_normalizer.

In [ ]:
def text_normalizer(text, sw_list):

    processed_text = []

    # Convierte el texto a minúsuculas y eliminación de espacios innecesarios, además de caracteres html
    text = text.strip().lower().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('<br /><br />', ' ')

    # Librería Beautififul Soup, para tener más seguridad de eliminar los html
    text = BeautifulSoup(text, "html5lib").get_text()

    # Eliminar caracteres "extraños"
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Tabla para eliminar signos de puntuación
    table = str.maketrans('', '', string.punctuation)

    # Segmentar texto en frases
    sentences = text.split('.')

    # Para cada frase, vamos palabra por palabra, eliminando los stop words, signos de puntuación,
    # y transformamos los números en palabras
    for sentence in sentences:
      words = sentence.split(' ')
      # Para cada palabra
      for word in words:

        if word not in string.punctuation:
          word = word.translate(table)

          if word not in sw_list:

            if word.isdigit():
              word = num2words(word, lang='en')
            processed_text.append(word)

    # retornamos el texto normalizado, cambiando los dobles espacios por espacios simples

    return ' '.join(processed_text).replace('  ', ' ')

Esta es la función de lemmatización. Las dos se pueden usar consecutivamente: primero la normalizer, y después, con el output de ésta, la de lemmatización.

In [ ]:
def lemmatizacion (text):
  text_lemma = []
  doc_1 = nlp_en(text)

  for idx, token in enumerate(doc_1):
    text_lemma.append(token.lemma_)

  text_lemmatized = ' '.join(text_lemma)

  return text_lemmatized

## Carga de datasets

Cargamos los tres datasets, y, para evitar problemas, hacemos un casting de las columas de texto a string.

In [9]:
dtype = {'text': str, 'title': str}

df_beauty_balanced = pd.read_csv('/content/drive/MyDrive/Practica_NLP_25/Project/df_beauty_balanced.csv', dtype = dtype, na_values = '')
df_health_balanced = pd.read_csv('/content/drive/MyDrive/Practica_NLP_25/Project/df_health_balanced.csv', dtype = dtype, na_values = '')
df_digital_balanced = pd.read_csv('/content/drive/MyDrive/Practica_NLP_25/Project/df_digital_balanced.csv', dtype = dtype, na_values = '')

In [10]:
df_beauty_balanced['title'] = df_beauty_balanced['title'].astype(str)
df_beauty_balanced['text'] = df_beauty_balanced['text'].astype(str)
df_health_balanced['title'] = df_health_balanced['title'].astype(str)
df_health_balanced['text'] = df_health_balanced['text'].astype(str)
df_digital_balanced['title'] = df_digital_balanced['title'].astype(str)
df_digital_balanced['text'] = df_digital_balanced['text'].astype(str)

Comprobamos, al menos en el caso del dataset de belleza, que tenemos nuestras 30.000 filas.

In [11]:
df_beauty_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   30000 non-null  object 
 1   text    30000 non-null  object 
 2   rating  30000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 703.3+ KB


## Pre-preprocesado y Stop Words

En este apartado vemos algunos aspectos previos a definir las funciones de preprocesado, siendo el más importante el definir la lista de Stop Words.

Probamos que podemos definir la tabla para quitar los signos de puntuación.

In [12]:
table = str.maketrans('', '', string.punctuation)

Teniendo en cuenta que tratamos con lengua inglesa, que no tiene acentos, quizás no es tan necesario, pero podemos comprobar que contamos con el método que lo hace.

In [13]:
text_with_accents = 'Téxtô Àcěntuado ñ'
unicodedata.normalize('NFKD', text_with_accents).encode('ascii', 'ignore').decode('utf-8', 'ignore')

'Texto Acentuado n'

Pasamos ahora a los stop words. Podemos cargar la lista que proporciona la librería stop_words en lengua inglesa.

In [14]:
sw_list = get_stop_words('en')


Miramos el tamaño de la lista, que da más de 1.300 elementos.

In [15]:
len(sw_list)

1333

El problema es que, muy fácilmente vemos que están incluidas palabras que encontramos en la visualización que eran muy importantes, como 'great', que lógicamente implica un sentimiento positivo. Por lo tanto, esta lista de palabras no es apropiada, por ser demasiada amplia.

In [16]:
'great' in sw_list

True

Probamos entonces la lista de palabras de SpaCy, que contiene una cantidad de palabra mucho más reducida, incluyendo solo, aparentemente, lo que realmente entendemos como stop words, que no contienen o reflejan contenido.

In [17]:
from spacy.lang.en.stop_words import STOP_WORDS

In [18]:
print(list(STOP_WORDS)[:20])

['anywhere', 'nowhere', 'and', 'hereafter', 'now', 'from', 'full', 'over', 'often', 'besides', 'else', 'up', '’ll', 'thru', 'became', 'because', 'twenty', 'than', 'sixty', 'last']


In [19]:
len(STOP_WORDS)

326

Como cabía esperar, la palabra 'great' no está incluida en la lista de stop words de Spacy, por lo que será esta la que utilicemos.

In [20]:
list_SW_Spacy = list(STOP_WORDS)

In [21]:
'great' in list_SW_Spacy

False

## Normalización sin lemmatización

Ya estamos listos para definir la función de normalización sin que incluya la lemmatización. Nos hemos basado en la pipeline presente en el notebook correspondiente de clase, aunque hemos hecho algunos cambios. La función sigue siendo aplicada a cada fila de una columna de dataframe, con una lista de stop words dada.

Hemos incluido una serie de replace() que quitan la mayor parte, si no todos, los carácteres de html. De hecho, el más común que hemos encontrado con diferencia es el cambio de párrafo. Para mayor seguridad en este aspecto, he aplicado además la libreria BeautifulSoup.

Hemos dividido en dos el proceso de quitar los signos de puntuación y tradudir los números en cifras a palabras. Prefiero controlar el proceso un poco mejor de esta forma.

También hemos cambiado el código original para que la función quite de veras las palabras que están en la lista de stop words.

Finalmente, quito los dobles espacios y los sustituyo por unos simples, como medida final de limpieza.

In [30]:
def text_normalizer(text, sw_list):

    processed_text = []

    # Convierte el texto a minúsuculas y eliminación de espacios innecesarios, además de caracteres html
    text = text.strip().lower().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('<br /><br />', ' ')

    # Librería Beautififul Soup, para tener más seguridad de eliminar los html
    text = BeautifulSoup(text, "html5lib").get_text()

    # Eliminar caracteres "extraños"
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Tabla para eliminar signos de puntuación
    table = str.maketrans('', '', string.punctuation)

    # Segmentar texto en frases
    sentences = text.split('.')

    # Para cada frase, vamos palabra por palabra, eliminando los stop words, signos de puntuación,
    # y transformamos los números en palabras
    for sentence in sentences:
      words = sentence.split(' ')
      # Para cada palabra
      for word in words:

        if word not in string.punctuation:
          word = word.translate(table)

          if word not in sw_list:

            if word.isdigit():
              word = num2words(word, lang='en')
            processed_text.append(word)

    # retornamos el texto normalizado, cambiando los dobles espacios por espacios simples

    return ' '.join(processed_text).replace('  ', ' ')

Podemos ver algún ejemplo, a ver cómo queda. sí parece que hace una normalización correcta.

In [31]:
text = df_beauty_balanced['text'][239]

In [32]:
text

'it’s an okay brush but that factory smell is strong and doesn’t go away'

In [33]:
text_normalizer(text, list_SW_Spacy)

'okay brush factory smell strong doesnt away'

Podemos ver que se eliminan esos molestos códigos de html, y se transforman los números en letras.

In [34]:
text = 'PROS:<br /><br />- The wand contains a rechargeable battery that last for 30 uses.'

In [35]:
text_normalizer(text, list_SW_Spacy)

'pros wand contains rechargeable battery thirty uses'

Otro ejemplo, un poco más largo de review. también parece que lo hace bien.

In [36]:
text_1 = 'This was a great product made to be helpful For moisturizing but, also being sensitive enough for use on a newborns skin. The baby i had ordered this for, has a unique combo- half Icelandic and half African… Creating skin mixture that needs very delicate and natural products that still can provide plenty of moisture.'

In [37]:
text_normalizer(text_1, list_SW_Spacy)

'great product helpful moisturizing sensitive use newborns skin baby ordered unique combo half icelandic half african creating skin mixture needs delicate natural products provide plenty moisture'

## Lemmatización

De cara a la lemmatización, comprobamos inicialmente que el modelo que hemos descargado de SpaCy funciona. Para ello partimos del texto text_1 de antes y lo dividimos en frases y en tokens con las funciones de SpaCy.-

In [38]:
text_1 = 'This was a great product made to be helpful For moisturizing but, also being sensitive enough for use on a newborns skin. The baby i had ordered this for, has a unique combo- half Icelandic and half African… Creating skin mixture that needs very delicate and natural products that still can provide plenty of moisture.'

In [39]:
doc = nlp_en(text_1)

In [40]:
for idx, sent in enumerate(doc.sents):
    print('Frase {0:5}{1:5}'.format(str(idx), sent.text))

Frase 0    This was a great product made to be helpful For moisturizing but, also being sensitive enough for use on a newborns skin.
Frase 1    The baby i had ordered this for, has a unique combo- half Icelandic and half African… Creating skin mixture that needs very delicate and natural products that still can provide plenty of moisture.


In [41]:
for idx, token in enumerate(doc):
    print('Token {0:5}{1:5}'.format(str(idx), token.text))

Token 0    This 
Token 1    was  
Token 2    a    
Token 3    great
Token 4    product
Token 5    made 
Token 6    to   
Token 7    be   
Token 8    helpful
Token 9    For  
Token 10   moisturizing
Token 11   but  
Token 12   ,    
Token 13   also 
Token 14   being
Token 15   sensitive
Token 16   enough
Token 17   for  
Token 18   use  
Token 19   on   
Token 20   a    
Token 21   newborns
Token 22   skin 
Token 23   .    
Token 24   The  
Token 25   baby 
Token 26   i    
Token 27   had  
Token 28   ordered
Token 29   this 
Token 30   for  
Token 31   ,    
Token 32   has  
Token 33   a    
Token 34   unique
Token 35   combo-
Token 36   half 
Token 37   Icelandic
Token 38   and  
Token 39   half 
Token 40   African
Token 41   …    
Token 42   Creating
Token 43   skin 
Token 44   mixture
Token 45   that 
Token 46   needs
Token 47   very 
Token 48   delicate
Token 49   and  
Token 50   natural
Token 51   products
Token 52   that 
Token 53   still
Token 54   can  
Token 55   provide
Toke

Ahora comprobamos que la lemmatizacion del texto funciona correctamente. Lo cierto es que no cambian muchas palabras, pero sí se observa que los tiempos verbales de 'to be', como 'being' o 'was' los cambia a 'be'. También elimina los plurales.

In [42]:
print('{0:10}{1:10}{2:10}'.format('Token', 'Lemma', 'PoS Tag'))
for idx, token in enumerate(doc):
    print('{0:10}{1:10}{2:10}'.format(token.text, token.lemma_, token.pos_))

Token     Lemma     PoS Tag   
This      this      PRON      
was       be        AUX       
a         a         DET       
great     great     ADJ       
product   product   NOUN      
made      make      VERB      
to        to        PART      
be        be        AUX       
helpful   helpful   ADJ       
For       for       ADP       
moisturizingmoisturizeVERB      
but       but       CCONJ     
,         ,         PUNCT     
also      also      ADV       
being     be        AUX       
sensitive sensitive ADJ       
enough    enough    ADV       
for       for       ADP       
use       use       NOUN      
on        on        ADP       
a         a         DET       
newborns  newborn   NOUN      
skin      skin      NOUN      
.         .         PUNCT     
The       the       DET       
baby      baby      NOUN      
i         I         PRON      
had       have      AUX       
ordered   order     VERB      
this      this      PRON      
for       for       ADP       
,     

Ya podemos definir la función de lemmatización, que es bastante sencillita, toma el texto inicial, lo pasa a un objeto con el modelo de inglés descargado, y va palabra a palabra aplicando la lemmatización.

In [43]:
def lemmatizacion (text):
  text_lemma = []
  doc_1 = nlp_en(text)

  for idx, token in enumerate(doc_1):
    text_lemma.append(token.lemma_)

  text_lemmatized = ' '.join(text_lemma)

  return text_lemmatized

Veamos como queda la lemmatización si la aplicamos a un texto.

In [44]:
text_1

'This was a great product made to be helpful For moisturizing but, also being sensitive enough for use on a newborns skin. The baby i had ordered this for, has a unique combo- half Icelandic and half African… Creating skin mixture that needs very delicate and natural products that still can provide plenty of moisture.'

Como cabía esperar, la lemmatización transforma los verbos en infinitivos, y, curiosamente, también el pronombre I en mayúsculas. Quizás es algo molesto el que, al tomar los signos de puntuación como tokens, nuestra función añada un espacio entre ellas.

In [45]:
lemmatizacion(text_1)

'this be a great product make to be helpful for moisturize but , also be sensitive enough for use on a newborn skin . the baby I have order this for , have a unique combo- half icelandic and half african … create skin mixture that need very delicate and natural product that still can provide plenty of moisture .'

Con todo, recordamos que aplicaremos la función de lemmatización a un texto ya normalizado, que es lo que comparamos a continuación. Vemos que los pronombres han sido ya excluidos, así como los signos de puntuación. La lemmatización se sigue aplicando a los verbos.

Lo cierto es que, sinceramente, no veo que haya muchos cambios debido a la lemmatización que pudiesen ser relevantes a la hora de estimar un sentimiento, aunque puedo estar equivocado.

In [46]:
text_normalizer(text_1, list_SW_Spacy)

'great product helpful moisturizing sensitive use newborns skin baby ordered unique combo half icelandic half african creating skin mixture needs delicate natural products provide plenty moisture'

In [47]:
lemmatizacion(text_normalizer(text_1, list_SW_Spacy))

'great product helpful moisturizing sensitive use newborn skin baby order unique combo half icelandic half african create skin mixture need delicate natural product provide plenty moisture'

## Preprocesado de los datasets

Ahora queda el prepocesar los datasets. Aplicaremos de forma secuancial las dos funciones tanto a las columnas de 'text' como a las de 'title'. Inicialmente almacenamos las columnas preprocesadas en el propio dataset.

Comenzamos con el dataset de belleza.

In [48]:
df_beauty_balanced['text_prep'] = df_beauty_balanced['text'].apply(lambda x: text_normalizer(x, list_SW_Spacy))

In [49]:
df_beauty_balanced['text_lemmatized'] = df_beauty_balanced['text_prep'].apply(lambda x: lemmatizacion(x))

In [50]:
df_beauty_balanced['title_prep'] = df_beauty_balanced['title'].apply(lambda x: text_normalizer(x, list_SW_Spacy))

In [51]:
df_beauty_balanced['title_lemmatized'] = df_beauty_balanced['title_prep'].apply(lambda x: lemmatizacion(x))

Ahora nos quedamos con las columnas ya prepocesadas, y evidentemente con la de rating

In [52]:
df_beauty_prep = df_beauty_balanced[['title_lemmatized', 'text_lemmatized', 'rating']]

In [53]:
df_beauty_prep.head()

,title_lemmatized,text_lemmatized,rating
0,look hook body wash holder,look hook body wash holder long time wish litt...,3.0
1,good shampoo date,nourish shampoo contain natural ingredient lot...,5.0
2,waste money,work remotely husband attempt use clean dry sk...,1.0
3,set spray,product effective set makeup actually cause sm...,3.0
4,product wonderful issue,deep steep vanilla peppermint wonderful fragra...,4.0


In [54]:
df_beauty_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title_lemmatized  30000 non-null  object 
 1   text_lemmatized   30000 non-null  object 
 2   rating            30000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 703.3+ KB


Guardamos el dataset preprocesado en otro fichero, en este caso llamado df_beauty_prep.

In [55]:
df_beauty_prep.to_csv('/content/drive/MyDrive/Practica_NLP_25/Project/df_beauty_prep.csv', index = False)

Aplicamos el mismo proceso al dataset de salud.

In [56]:
df_health_balanced['text_prep'] = df_health_balanced['text'].apply(lambda x: text_normalizer(x, list_SW_Spacy))

/tmp/ipython-input-3794127855.py:8: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  text = BeautifulSoup(text, "html5lib").get_text()


In [57]:
df_health_balanced['text_lemmatized'] = df_health_balanced['text_prep'].apply(lambda x: lemmatizacion(x))

In [58]:
df_health_balanced['title_prep'] = df_health_balanced['title'].apply(lambda x: text_normalizer(x, list_SW_Spacy))

In [59]:
df_health_balanced['title_lemmatized'] = df_health_balanced['title_prep'].apply(lambda x: lemmatizacion(x))

In [60]:
df_health_prep = df_health_balanced[['title_lemmatized', 'text_lemmatized', 'rating']]

In [61]:
df_health_prep.head()

,title_lemmatized,text_lemmatized,rating
0,recommend buying scale,receive scale weigh weigh second time pound di...,1.0
1,life extension product slightly price,life extension product slightly price three th...,5.0
2,fire hazard beware,fire hazard battery heat,1.0
3,good flavor,like vanilla chocolate flavor floss lovecocofl...,2.0
4,phlegm clearing throat relief,help throat phlegm prevent clearing throat tho...,1.0


In [62]:
df_health_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title_lemmatized  30000 non-null  object 
 1   text_lemmatized   30000 non-null  object 
 2   rating            30000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 703.3+ KB


In [63]:
df_health_prep.to_csv('/content/drive/MyDrive/Practica_NLP_25/Project/df_health_prep.csv', index = False)

Terminamos el notebook preprocesando el dataset de música digital.

In [64]:
df_digital_balanced['text_prep'] = df_digital_balanced['text'].apply(lambda x: text_normalizer(x, list_SW_Spacy))

In [65]:
df_digital_balanced['text_lemmatized'] = df_digital_balanced['text_prep'].apply(lambda x: lemmatizacion(x))

In [66]:
df_digital_balanced['title_prep'] = df_digital_balanced['title'].apply(lambda x: text_normalizer(x, list_SW_Spacy))

In [67]:
df_digital_balanced['title_lemmatized'] = df_digital_balanced['title_prep'].apply(lambda x: lemmatizacion(x))

In [69]:
df_digital_prep = df_digital_balanced[['title_lemmatized', 'text_lemmatized', 'rating']]

In [70]:
df_digital_prep.head()

,title_lemmatized,text_lemmatized,rating
0,rip,description clearly state factory seal cd nice...,1.0
1,classic 90 new age wolf music,new age wolf surprisingly good new age album f...,4.0
2,great 80 hard rock sound,good tune,4.0
3,great music great movie gouge price,do not wrong love cd seller vulture buy cd itu...,1.0
4,like alot,like alot there s definitely iron maiden influ...,4.0


In [71]:
df_digital_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title_lemmatized  30000 non-null  object 
 1   text_lemmatized   30000 non-null  object 
 2   rating            30000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 703.3+ KB


In [72]:
df_digital_prep.to_csv('/content/drive/MyDrive/Practica_NLP_25/Project/df_digital_prep.csv', index = False)

Fin del ejercicio 2.